In [27]:
import os
import pandas as pd

from mosekTools.solver import lsq_pos as ll


def lsq_pos(matrix, rhs):
    return pd.Series(index=matrix.columns,
                     data=ll(matrix.values, rhs.values))


def Sharpe_Ratio(ts):
    return 16 * ts.mean() / ts.std()

In [28]:
# load data from csv file
data = pd.read_csv(os.path.join("data", "data.csv"), index_col=0, parse_dates=True).ffill()
returns = data.pct_change().fillna(0.0)

stocks = ["GOOG", "T", "AAPL", "GS", "IBM"]
index = "^GSPC"

In [30]:
# construct a rhs
rhsZero = pd.Series(index=data.index, data=0.0)

wMin = lsq_pos(matrix=data[stocks].pct_change().fillna(0.0), rhs=rhsZero)
wTrack = lsq_pos(matrix=data[stocks], rhs=data[index])

d = dict()
d["Min Variance"] = (returns[stocks] * wMin).sum(axis=1)
d["Index"] = returns[index]
d["1/N"] = returns[stocks].mean(axis=1)
d["Tracking"] = (returns[stocks] * wTrack).sum(axis=1)
frame = pd.DataFrame(d)

# apply some simple diagnostics
print("Annualized Sharpe ratio")
print(frame.apply(Sharpe_Ratio))
print("Standard deviation of returns")
print(frame.std())

Annualized Sharpe ratio
1/N             0.895113
Index           0.841828
Min Variance    0.857319
Tracking        0.580367
dtype: float64
Standard deviation of returns
1/N             0.010229
Index           0.010035
Min Variance    0.008710
Tracking        0.015660
dtype: float64
